<a href="https://colab.research.google.com/github/eyessoo/sql/blob/main/SQL_Programming_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# We will first load the sql extension into our environment
#
%load_ext sql

# Then connect to our in memory sqlite database
# NB: This database will cease to exist as soon as the database connection is closed
#
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @None'

In [ ]:
# Importing the python libraries that we will need in this notebook
#
import csv
import pandas as pd

In [ ]:
# Loading the first table from a local CSV file
#
with open('Uganda_Karamoja_Subcounty_Crop_Yield_Population.csv','r') as f:
    ds = pd.read_csv(f, index_col=0, encoding='utf-8') 

# Then storing it in an SQL table of our in memory sqlite database
%sql DROP TABLE if EXISTS ds;
%sql PERSIST ds;

 * sqlite://
Done.
 * sqlite://


'Persisted ds'

In [ ]:
# Displaying 5 rows in the dataset

%%sql
select * from ds limit 5;

 * sqlite://
Done.


OBJECTID,SUBCOUNTY_NAME,DISTRICT_NAME,POP,Area,Karamoja,S_Yield_Ha,M_Yield_Ha,Crop_Area_Ha,S_Area_Ha,M_Area_Ha,S_Prod_Tot,M_Prod_Tot
263,KACHERI,KOTIDO,17244,1067176155,Y,354.20741069999997,1137.467019,7023.533691,6434.342449,528.1242294,2279091.779,600723.8929
264,KOTIDO,KOTIDO,52771,597575188,Y,367.89052319999996,1162.9966869999998,13587.990759999999,12455.59264,824.7670809,4582294.492,959201.3825
265,KOTIDO TOWN COUNCIL,KOTIDO,27389,23972401,Y,369.3141771,1167.005832,1656.5318550000002,1520.322052,8.561643817,561476.4874,9991.488268000001
266,NAKAPERIMORU,KOTIDO,38775,419111591,Y,283.3245689,852.3665784,7087.823334000001,6761.488901000001,45.72171184,1915695.928,38971.65908
267,PANYANGARA,KOTIDO,65704,880955930,Y,373.8369255,1283.859882,10398.24939,10111.198129999999,172.6119139,3779939.224,221609.5114


In [ ]:
#1 Display a list of Sub Counties and their population and areas.

%%sql
select SUBCOUNTY_NAME, POP, Area from ds limit 5;



 * sqlite://
Done.


SUBCOUNTY_NAME,POP,Area
KACHERI,17244,1067176155
KOTIDO,52771,597575188
KOTIDO TOWN COUNCIL,27389,23972401
NAKAPERIMORU,38775,419111591
PANYANGARA,65704,880955930


In [ ]:
#Sort the list of districts by total crop area (descending order).

%%sql
SELECT DISTRICT_NAME, Crop_Area_Ha from ds
GROUP BY DISTRICT_NAME
ORDER BY Crop_Area_Ha desc;



 * sqlite://
Done.


DISTRICT_NAME,Crop_Area_Ha
KOTIDO,13278.52043
NAPAK,6471.047334000001
MOROTO,2217.2907170000003
NAKAPIRIPIRIT,1161.3902289999999
KAABONG,957.4939457999999
ABIM,792.8987889
AMUDAT,60.77208552


In [ ]:
#Select only the Sub counties from the Moroto district, order them alphabetically and show their production of sorghum.


%%sql
SELECT SUBCOUNTY_NAME, S_Area_Ha FROM ds
WHERE DISTRICT_NAME = "MOROTO"
ORDER BY SUBCOUNTY_NAME ASC;

 * sqlite://
Done.


SUBCOUNTY_NAME,S_Area_Ha
KATIKEKILE,30.15704817
NADUNGET,2406.1906649999996
NORTHERN DIVISION,0.130940756
RUPA,1989.1195079999998
SOUTHERN DIVISION,0.17139046100000002
TAPAC,315.979223


In [ ]:
#Compute the total Maize production per District.

%%sql
SELECT DISTRICT_NAME, M_Prod_Tot from ds
GROUP BY DISTRICT_NAME;



 * sqlite://
Done.


DISTRICT_NAME,M_Prod_Tot
ABIM,258010.2444
AMUDAT,2081.356199
KAABONG,136265.6835
KOTIDO,179681.139
MOROTO,155838.875
NAKAPIRIPIRIT,1198586.875
NAPAK,414509.6206


In [ ]:
#Compute the number of Sub counties where Maize is produced and the total Maize production per District.

%%sql
SELECT DISTRICT_NAME, M_Prod_Tot, COUNT(SUBCOUNTY_NAME) FROM ds
WHERE M_Prod_Tot > 0
GROUP BY DISTRICT_NAME;


 * sqlite://
Done.


DISTRICT_NAME,M_Prod_Tot,COUNT(SUBCOUNTY_NAME)
ABIM,258010.2444,6
AMUDAT,2081.356199,4
KAABONG,136265.6835,14
KOTIDO,179681.139,6
MOROTO,155838.875,5
NAKAPIRIPIRIT,1198586.875,8
NAPAK,414509.6206,8


In [ ]:
#Compute the overall Crop area in all Sub counties where population is over 20000.

%%sql
SELECT DISTRICT_NAME, total(Crop_Area_Ha) FROM ds
WHERE POP > 20000
GROUP BY DISTRICT_NAME
ORDER BY DISTRICT_NAME ASC; 


 * sqlite://
Done.


DISTRICT_NAME,total(Crop_Area_Ha)
AMUDAT,5704.671632
KAABONG,26629.900721300004
KOTIDO,46009.115769
MOROTO,5809.076824
NAKAPIRIPIRIT,19909.559379
NAPAK,16150.565830000001


In [ ]:
#Sort the Maize production in descending order by Districts, only taking into account Sub counties 
# where Maize area is larger than Sorghum area, and display the number of Sub counties per district matching that criteria.



%%sql
SELECT DISTRICT_NAME, COUNT(SUBCOUNTY_NAME), M_Prod_Tot FROM ds
WHERE M_Area_Ha > S_Area_Ha
GROUP BY DISTRICT_NAME
ORDER BY DISTRICT_NAME DESC;




 * sqlite://
Done.


DISTRICT_NAME,COUNT(SUBCOUNTY_NAME),M_Prod_Tot
NAPAK,2,291177.2726
NAKAPIRIPIRIT,3,1198586.875
MOROTO,3,46.22440465
KAABONG,3,934881.6576
AMUDAT,1,1340672.952
